In [ ]:
# Box Office Mojo - daily data on top wide releases yearly in 2021 and 2022

In [1]:
import subprocess
import sys
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])# import data libraries
    
install("xlwt")
install("openpyxl")
install("selenium")
install("webdriver-manager")

In [2]:

import pandas as pd
import numpy as np
from datetime import date, datetime
from time import sleep
import pytz
import random
from openpyxl import *
from openpyxl.utils.dataframe import dataframe_to_rows
from string import *


# import selenium/webscraping libs
from urllib.request import urlopen
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

# Access webdriver for Chrome
driver_path = '/Users/zacharywong/Downloads/chromedriver'
service = Service(driver_path)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
sleepTime = random.randrange(3, 4)

# excel
path = "/Users/zacharywong/github/zacharywong2023/BoxOffice/"
totalFileName = "DecayCurveData.xlsx"
wb = Workbook()

# keep track of day/time
est = pytz.timezone('US/Eastern')
fmt = '%d/%m/%Y %H:%M:%S'
now = datetime.now()
now = now.astimezone(est).strftime(fmt)
print("day/time: " + now)

urlYearly = 'https://www.boxofficemojo.com/year/?ref_=bo_nb_hm_secondarytab'
dailyColumns = ["Date", "DOW", "Rank", "Daily", "%+/-YR", "%+/-LW", "Theaters", "Avg", "To Date", "Day", "Title", "Genre", "Distributor"]
yearlyColumns = ["Rank", "Release", "Gross", "Max Th", "Opening", "% of Total", "Open Th", "Open", "Close", "Distributor"]

dfDailyTable = pd.DataFrame()
dfYearlyTable = pd.DataFrame()

years = [2022, 2021]




====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [/Users/zacharywong/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache


day/time: 08/07/2022 21:32:55


In [3]:
def writetoExcel(df, sheetName):
    ws = wb.create_sheet()
    ws.title = sheetName
    rows = dataframe_to_rows(df, index=False, header=True)
    for row in rows:
        ws.append(row)
    wb.save(totalFileName)
        
 

In [4]:
# Filter by In Year Releases and Wide Releases
def clickFilters():
    XPATHinYear= "/html/body/div[1]/main/div/div/div[1]/div[2]/span/form/span/select"
    XPATHwideRelease = "/html/body/div[1]/main/div/div/div[1]/div[3]/span/form/span/select"
    
    try:
        filterInYearSelect = Select(driver.find_element(By.XPATH, XPATHinYear))
    except:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATHinYear)))
        filterInYearSelect = Select(driver.find_element(By.XPATH, XPATHinYear))
        
    filterInYearSelect.select_by_visible_text("In-year releases")
    sleep(sleepTime)
    
    try:
        filterwidereleaseSelect = Select(driver.find_element(By.XPATH, XPATHwideRelease))
    except:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATHwideRelease)))
        filterwidereleaseSelect = Select(driver.find_element(By.XPATH, XPATHwideRelease))
        
    filterwidereleaseSelect.select_by_visible_text("Wide releases")
        

In [5]:
def getMovieLink(index):
    #//*[@id="table"]/div/table[2]/tbody/tr[2]/td[2]/a
    XPATHMovie = "//*[@id='table']/div/table[2]/tbody/tr[{index}]/td[2]/a".format(index = index)
    try: 
        movie = driver.find_element(By.XPATH, XPATHMovie) 
    except: 
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATHMovie)))
        movie = driver.find_element(By.XPATH, XPATHMovie)
    return movie, movie.text

In [6]:
def getRowColumnNumber():
    adjustForHeader = 1
    rowXPATH = "//*[@id='table']/div/table[2]/tbody/tr"
    colXPATH = "//*[@id='table']/div/table[2]/tbody/tr[1]/th"
    colNumber = 0
    rowNumber = len(driver.find_elements(By.XPATH, rowXPATH))-adjustForHeader
    return rowNumber, colNumber


In [7]:
def getTableData():
    tempResult = pd.read_html(driver.page_source)[1]
    return tempResult

In [8]:
def cleanTable(df):
    for column in df:
        df[column] = df[column].replace(['-'], np.nan)
        #if df[column].dtypes == bool:
        #    df.drop(column, inplace = True, axis = 1)
    df = df.dropna(axis = 1, how = 'all')
    df = df.drop(df.columns[[10]], axis = 1)
    return df

In [9]:
def renameYearlyTable(df):
    df.columns = yearlyColumns
    return df

In [10]:
def renameDailyTable(df):
    df.columns = dailyColumns
    return df

In [11]:
def reorderDailyTable(df):
    colsOrdered = ["Distributor", "Title", "Genre", "Date", "DOW", "Rank", "Daily", "%+/-YR", "%+/-LW", "Theaters", "Avg", "To Date", "Day"]
    df = df[colsOrdered]
    return df

In [12]:
def cleanDailyDistributor(df):
    df["Distributor"] = df["Distributor"].str.split('\n').str[0]
    return df

In [13]:
def cleanDate(df):
    df["Date"] = df["Date"].str.split('COVID-19 Pandemic').str[0]
    return df

In [14]:
def getDailyMovieData(movieTitle):
    rowNumber, colNumber = getRowColumnNumber()
    indexList = []
    XPATHGenre = "//*[@id='a-page']/main/div/div[3]/div[4]/div[6]/span[2]"
    XPATHDistributor = "//*[@id='a-page']/main/div/div[3]/div[4]/div[1]/span[2]"
    #            value = (driver.find_element(By.XPATH, XPATHValue)).text
    for number in range(0, rowNumber):
        indexList.append(number)
    
    tempResult = pd.DataFrame(index = indexList)
    tempResult = getTableData()
    tempResult["Title"] = movieTitle
    tempResult["Genre"] = (driver.find_element(By.XPATH, XPATHGenre)).text
    tempResult["Distributor"] = (driver.find_element(By.XPATH, XPATHDistributor)).text
    adjustRow = 2
    adjustCol = 1
    #display(tempResult)
    #for row in range(0, rowNumber):
    #    for col in range(0, colNumber-1+2):
            #print(rowNumber, columns[colNumber])
            #print(row, col)
            #print(rowNumber, colNumber)
    #        if col == 0:
    #            value = movieTitle
    #        elif col == 1: #//*[@id="a-page"]/main/div/div[3]/div[4]/div[6]/span[2]
    #            XPATHValue = "//*[@id='a-page']/main/div/div[3]/div[4]/div[6]/span[2]"
    #            value = (driver.find_element(By.XPATH, XPATHValue)).text
     #       elif col == 2 or col == 3:
     #           XPATHValue = "//*[@id='table']/div/table[2]/tbody/tr[{row}]/td[{col}]/a".format(row = row+adjustRow, col = col-1)
    #            value = (driver.find_element(By.XPATH, XPATHValue)).text
    #        else:
    #            XPATHValue = "//*[@id='table']/div/table[2]/tbody/tr[{row}]/td[{col}]".format(row = row+adjustRow, col = col-1)
    #            value = (driver.find_element(By.XPATH, XPATHValue)).text
    #        #print(rowNumber, columns[colNumber])
    #        tempResult.at[row, columns[col]] = value
    #        #print(tempResult)
    return tempResult
        
                

In [15]:
def getAllDailyData(rowNumber):
    print("Number of Movies:" + str(rowNumber))
    index = 2
    while index < rowNumber+2:
        print("index: " + str(index))
        startNow = datetime.now()
        movieDriver, movieTitle = getMovieLink(index)
        print(movieTitle)
        movieDriver.click()
        getRowColumnNumber()
        try:
            tempResult = getDailyMovieData(movieTitle)
        except:
            print("cannot read movies further - most likely website overload")
        global dfDailyTable
        dfDailyTable = dfDailyTable.append(tempResult)
        index += 1
        
        sleep(sleepTime)
        driver.execute_script("window.history.go(-1)")
        sleep(sleepTime/2)
        endNow = datetime.now()
        timeElapsed = endNow - startNow
        print("time elapsed: " + str(timeElapsed))


In [16]:
def runYear(year):
    #print(result)
    XPATH_Year = "//*[@class='a-link-normal' and text() = '{year}']".format(year = year)
    try:
        yearListDriver = driver.find_element(By.XPATH, XPATH_Year)
    except:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATH_Year)))
        yearListDriver = driver.find_element(By.XPATH, XPATH_Year)
    yearListDriver.click()
    clickFilters()
    rowNumber, colNumber = getRowColumnNumber()
    
    global dfYearlyTable
    dfYearlyTable = getTableData()
    dfYearlyTable = cleanTable(dfYearlyTable)
    dfYearlyTable = renameYearlyTable(dfYearlyTable)
    
    display(dfYearlyTable)
    
    sheetName = "{year}Summary.xlsx".format(year = year) 
    writetoExcel(dfYearlyTable, sheetName)
    getAllDailyData(rowNumber)
    

In [17]:
def run():
    for year in years:
        try:
            driver.get(urlYearly)
        except: 
            print("Cannot access {url}...quitting now").format(url = urlYearly)
        #XPATH = "/html/body/div[1]/main/div/div/div[3]/div/table[2]/tbody/tr[2]/td[1]/a"
        runYear(year)
        sleep(sleepTime)
        driver.execute_script("window.history.go(-1)")
        
    global dfDailyTable
    display(dfDailyTable)
    sheetName = "DailyMovieData"
    writetoExcel(dfDailyTable, sheetName)
    dfDailyTable = cleanTable(dfDailyTable)
    dfDailyTable = renameDailyTable(dfDailyTable)
    dfDailyTable = reorderDailyTable(dfDailyTable)
    dfDailyTable = cleanDailyDistributor(dfDailyTable)
    dfDailyTable = cleanDate(dfDailyTable)

    display(dfDailyTable)
    
    writetoExcel(dfDailyTable, sheetName)
    wb.save(filename = totalFileName)

In [18]:
startNow = datetime.now()
run()
endNow = datetime.now()
elapsedTime = endNow - startNow 
print("Total Time Elapsed: " + str(elapsedTime))
#result.head()

,Rank,Release,Gross,Max Th,Opening,% of Total,Open Th,Open,Close,Distributor
0,1,Top Gun: Maverick,"$581,906,113",4751,"$126,707,459",21.8%,4735,May 27,NaN,Paramount Pictures
1,2,Doctor Strange in the Multiverse of Madness,"$410,800,441",4534,"$187,420,998",45.6%,4534,May 6,NaN,Walt Disney Studios Motion Pictures
2,3,The Batman,"$369,345,583",4417,"$134,008,624",36.3%,4417,Mar 4,NaN,Warner Bros.
3,4,Jurassic World Dominion,"$341,915,830",4697,"$145,075,625",42.4%,4676,Jun 10,NaN,Universal Pictures
4,5,Sonic the Hedgehog 2,"$190,872,904",4258,"$72,105,176",37.8%,4234,Apr 8,Jul 8,Paramount Pictures
5,6,Minions: The Rise of Gru,"$164,528,580",4427,"$107,010,140",65%,4391,Jul 1,NaN,Universal Pictures
6,7,Uncharted,"$148,610,481",4275,"$44,010,155",29.6%,4275,Feb 18,NaN,Sony Pictures Entertainment (SPE)
7,8,Lightyear,"$109,422,200",4255,"$50,577,961",46.2%,4255,Jun 17,NaN,Walt Disney Studios Motion Pictures
8,9,The Lost City,"$105,344,029",4283,"$30,453,269",28.9%,4253,Mar 25,Jun 23,Paramount Pictures
9,10,The Bad Guys,"$96,137,975",4042,"$23,950,245",24.9%,4008,Apr 22,NaN,Universal Pictures


Number of Movies:42
index: 2
Top Gun: Maverick
time elapsed: 0:00:06.694669
index: 3
Doctor Strange in the Multiverse of Madness
time elapsed: 0:00:05.820972
index: 4
The Batman
time elapsed: 0:00:05.591477
index: 5
Jurassic World Dominion
time elapsed: 0:00:06.368810
index: 6
Sonic the Hedgehog 2
time elapsed: 0:00:06.134055
index: 7
Minions: The Rise of Gru
time elapsed: 0:00:05.463053
index: 8
Uncharted
time elapsed: 0:00:05.955525
index: 9
Lightyear
time elapsed: 0:00:05.300015
index: 10
The Lost City
time elapsed: 0:00:06.231403
index: 11
The Bad Guys
time elapsed: 0:00:05.584872
index: 12
Fantastic Beasts: The Secrets of Dumbledore
time elapsed: 0:00:05.468581
index: 13
Scream
time elapsed: 0:00:05.444080
index: 14
Elvis
time elapsed: 0:00:06.023047
index: 15
Morbius
time elapsed: 0:00:05.473004
index: 16
Everything Everywhere All at Once
time elapsed: 0:00:06.241863
index: 17
Dog
time elapsed: 0:00:05.453424
index: 18
Jackass Forever
time elapsed: 0:00:06.331847
index: 19
The Bl

,Rank,Release,Gross,Max Th,Opening,% of Total,Open Th,Open,Close,Distributor
0,1,Spider-Man: No Way Home,"$804,793,477",4336,"$260,138,569",32.3%,4336,Dec 17,Jun 30,Sony Pictures Entertainment (SPE)
1,2,Shang-Chi and the Legend of the Ten Rings,"$224,543,292",4300,"$75,388,688",33.6%,4300,Sep 3,NaN,Walt Disney Studios Motion Pictures
2,3,Venom: Let There Be Carnage,"$213,550,366",4225,"$90,033,210",42.2%,4225,Oct 1,Jan 27,Sony Pictures Entertainment (SPE)
3,4,Black Widow,"$183,651,655",4275,"$80,366,312",43.8%,4160,Jul 9,NaN,Walt Disney Studios Motion Pictures
4,5,F9: The Fast Saga,"$173,005,945",4203,"$70,043,165",40.5%,4179,Jun 25,NaN,Universal Pictures
...,...,...,...,...,...,...,...,...,...,...
74,75,Here Today,"$2,807,494",1230,"$1,042,171",37.1%,1200,May 7,NaN,Stage 6 Films
75,76,Finding You,"$2,753,985",1447,"$920,476",33.4%,1312,May 14,NaN,Roadside Attractions
76,77,Land,"$2,577,830",1349,"$899,810",34.9%,1231,Feb 12,NaN,Focus Features
77,78,Profile,"$1,744,740",2104,"$730,290",41.9%,2033,May 14,NaN,Focus Features


Number of Movies:79
index: 2
Spider-Man: No Way Home
time elapsed: 0:00:07.215804
index: 3
Shang-Chi and the Legend of the Ten Rings
time elapsed: 0:00:06.464372
index: 4
Venom: Let There Be Carnage
time elapsed: 0:00:06.456987
index: 5
Black Widow
time elapsed: 0:00:06.436028
index: 6
F9: The Fast Saga
time elapsed: 0:00:06.364592
index: 7
Eternals
time elapsed: 0:00:06.687664
index: 8
Sing 2
time elapsed: 0:00:10.355376
index: 9
No Time to Die
time elapsed: 0:00:06.316639
index: 10
A Quiet Place Part II
time elapsed: 0:00:06.432030
index: 11
Ghostbusters: Afterlife
time elapsed: 0:00:06.404530
index: 12
Free Guy
time elapsed: 0:00:06.513217
index: 13
Jungle Cruise
time elapsed: 0:00:06.534611
index: 14
Dune
time elapsed: 0:00:06.640088
index: 15
Godzilla vs. Kong
time elapsed: 0:00:10.150970
index: 16
Encanto
time elapsed: 0:00:06.553718
index: 17
Halloween Kills
time elapsed: 0:00:06.429273
index: 18
Cruella
time elapsed: 0:00:06.548989
index: 19
Space Jam: A New Legacy
time elapsed

,0,1,2,3,4,5,6,7,8,9,10,Title,Genre,Distributor
0,May 27,Friday,1,"$52,012,661",-,-,4735,"$10,984","$52,012,661",1,False,Top Gun: Maverick,Action Drama,Paramount Pictures\nSee full company information
1,May 28,Saturday,1,"$38,017,701",-26.9%,-,4735,"$8,029","$90,030,362",2,False,Top Gun: Maverick,Action Drama,Paramount Pictures\nSee full company information
2,May 29,Sunday,1,"$36,677,097",-3.5%,-,4735,"$7,745","$126,707,459",3,False,Top Gun: Maverick,Action Drama,Paramount Pictures\nSee full company information
3,May 30Memorial Day,Monday,1,"$33,807,521",-7.8%,-,4735,"$7,139","$160,514,980",4,False,Top Gun: Maverick,Action Drama,Paramount Pictures\nSee full company information
4,May 31,Tuesday,1,"$15,800,746",-53.3%,-,4735,"$3,337","$176,315,726",5,False,Top Gun: Maverick,Action Drama,Paramount Pictures\nSee full company information
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,Dec 19,Sunday,17,"$12,962",-2.6%,-80.9%,898,$14,"$455,492",10,False,National Champions,Drama Sport,STX Entertainment\nSee full company information
10,Dec 20,Monday,16,"$8,208",-36.7%,-74.1%,898,$9,"$463,700",11,False,National Champions,Drama Sport,STX Entertainment\nSee full company information
11,Dec 21,Tuesday,16,"$9,429",+14.9%,-79%,898,$10,"$473,129",12,False,National Champions,Drama Sport,STX Entertainment\nSee full company information
12,Dec 22,Wednesday,23,"$1,244",-86.8%,-95.9%,898,$1,"$474,373",13,False,National Champions,Drama Sport,STX Entertainment\nSee full company information


,Distributor,Title,Genre,Date,DOW,Rank,Daily,%+/-YR,%+/-LW,Theaters,Avg,To Date,Day
0,Paramount Pictures,Top Gun: Maverick,Action Drama,May 27,Friday,1,"$52,012,661",NaN,NaN,4735,"$10,984","$52,012,661",1
1,Paramount Pictures,Top Gun: Maverick,Action Drama,May 28,Saturday,1,"$38,017,701",-26.9%,NaN,4735,"$8,029","$90,030,362",2
2,Paramount Pictures,Top Gun: Maverick,Action Drama,May 29,Sunday,1,"$36,677,097",-3.5%,NaN,4735,"$7,745","$126,707,459",3
3,Paramount Pictures,Top Gun: Maverick,Action Drama,May 30Memorial Day,Monday,1,"$33,807,521",-7.8%,NaN,4735,"$7,139","$160,514,980",4
4,Paramount Pictures,Top Gun: Maverick,Action Drama,May 31,Tuesday,1,"$15,800,746",-53.3%,NaN,4735,"$3,337","$176,315,726",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,STX Entertainment,National Champions,Drama Sport,Dec 19,Sunday,17,"$12,962",-2.6%,-80.9%,898,$14,"$455,492",10
10,STX Entertainment,National Champions,Drama Sport,Dec 20,Monday,16,"$8,208",-36.7%,-74.1%,898,$9,"$463,700",11
11,STX Entertainment,National Champions,Drama Sport,Dec 21,Tuesday,16,"$9,429",+14.9%,-79%,898,$10,"$473,129",12
12,STX Entertainment,National Champions,Drama Sport,Dec 22,Wednesday,23,"$1,244",-86.8%,-95.9%,898,$1,"$474,373",13


Total Time Elapsed: 0:13:52.986949
